In [1]:
import re

input_file = "input_files/day_12.txt"

with open(input_file) as lines:
    data = [l.strip() for l in lines]

In [6]:
rx = re.compile(r"#+")

def parse_lines(data):
    lines = []
    for line in data:
        pattern, counts = line.split()
        counts = tuple([int(n) for n in counts.split(',')])
        lines.append((pattern, counts))
    return lines
    
lines = parse_lines(data)   
len(lines), lines[0]

(1000, ('.?????...?', (1, 1, 1)))

# Part One

The basic idea is to consume each count by matching a string like ".###." (if count is three) onto the pattern. For each match recurse on the rest of the pattern and remaining counts. Part two is addressed by memoizing with the LRU cache.

In [3]:
from functools import cache

@cache
def find_matches(sub, counts, c=0):
    #print(sub, counts, c)
    if len(counts) == 0:
        ## If there no remaining count and not remaining'#' 
        ## in the string, this one worked
        if '#' not in sub: # this was a good match other wise it wasn't
            return 1
        # otherise it did not
        return 0
        
    # if the remaining string is too short don't bother continuing
    min_string = sum(counts) + len(counts) + 1 
    if len(sub) < min_string:
        return 0

    n, *counts = counts

    m = '.' + '#' * n + '.'

    # Each count n should represent n '#' characters 
    # Which must match a string like .###. in the input
    # Scan over the string until the point where no 
    # more matches are possible.
    # If a match is found, add the remaining string 
    # and counts to the stack for further processing
    
    cc = 0
    for i in range(len(sub) - (min_string - n) ):
        test_string = sub[i:i+n+2]
        remaining = '.' + sub[i+n+2:]

        # if we find a sub string with # in it,
        # it either needs to consume the count
        # or it needs to have a ? in it.
        # A string like #.# and a count of 3 means it
        # can't match

        if test_string[1:1+n] == '#' * n:
            #print("here", test_string[1:1+n])
            if test_string[-1] != "#":
                cc += find_matches(remaining, tuple(counts), c)
            break
        else:
            if sub[i] == '#':
                # this is passing over a non
                break
            if all(b=='?' or a==b for a, b in zip(m, sub[i:i+n+2])):
                remaining = '.'+sub[i+n+2:]
                cc += find_matches(remaining, tuple(counts), c)
        

    return cc


In [4]:
sum(find_matches('.'+match[0]+'.', tuple(match[1])) for match in lines)

7922

# Part Two

In [5]:
def unfold(pattern, counts):
    return ('.'+'?'.join([pattern] * 5) + '.', counts * 5)

sum(find_matches(*(unfold(*match))) for match in lines)


18093821750095